In [ ]:
import ipywidgets as widgets
import pyvista
import numpy as np
from IPython.display import display, clear_output
from tvbwidgets.ui.connectivity_matrix_editor_widget import ConnectivityMatrixEditor
from tvbwidgets.api import ConnectivityWidget
from tvb.datatypes.connectivity import Connectivity

# Load and configure connectivity object
conn = Connectivity().from_file()
conn.configure()

# Custom editor to wrap and enhance TVB's connectivity matrix editor
class CustomMatrixEditor(widgets.VBox, ConnectivityMatrixEditor):
    def __init__(self, connectivity, viewer, **kwargs):
        ConnectivityMatrixEditor.__init__(self, connectivity, **kwargs)
        widgets.VBox.__init__(self)
        self.viewer = viewer
        self.output = widgets.Output()
        
        with self.output:
            self.display()
        
        self.children = [self.output]

    # Handle cell clicks and update viewer with selected centres
    def on_cell_clicked(self, x, y, matrix_name):
        super().on_cell_clicked(x, y, matrix_name)
        if hasattr(self.viewer, "highlight_centres"):
            self.viewer.highlight_centres(self.selected_centres)

# Custom viewer to render connectivity in 3D and highlight selected regions
class CustomViewer(ConnectivityWidget):  
    def __init__(self, connectivity, width=600, height=600):
        super().__init__(connectivity, default_active_tab='viewers', width=width, height=height)
        self.highlight_output = widgets.Output()
        self.children = list(self.children) + [self.highlight_output]

    # Highlight selected centres in the 3D viewer
    def highlight_centres(self, selected_centres):
        with self.highlight_output: 
            clear_output(wait=True)
            connectivity_3d_viewer = self.viewers_tab.children[1]
            if hasattr(connectivity_3d_viewer, "output"):
                plotter = connectivity_3d_viewer.output.plotter
                if hasattr(self, "highlight_actor") and self.highlight_actor: 
                    plotter.remove_actor(self.highlight_actor, reset_camera=False)
                points = np.array(selected_centres)
                mesh_points = pyvista.PolyData(points)  
                points_size = connectivity_3d_viewer.output.CONFIG.point_size + 5
                self.highlight_actor = plotter.add_points(mesh_points, color="red", point_size=points_size)
                connectivity_3d_viewer.output.update_plot()

    # Display the viewer widget
    def custom_display(self):
        display(self)

# Initialize custom viewer and matrix editor widgets
custom_viewer = CustomViewer(conn, width=600, height=600)
custom_matrix_editor = CustomMatrixEditor(conn, viewer=custom_viewer)

# Display the widgets side by side
side_by_side = widgets.HBox(
    [custom_matrix_editor, custom_viewer],
    layout=widgets.Layout(
        width='100%',
        justify_content='space-between',
        padding='10px',
        border='1px solid #eee'
    )
)

display(side_by_side)



07-04-2025 01:29:13 - INFO - tvbwidgets - Version: 2.2.1
Using matplotlib as 2D backend.
2025-04-07 01:29:41,766 - INFO - tvb.storage.h5.encryption.data_encryption_handler - Cannot import syncrypto library.
07-04-2025 01:29:42 - INFO - tvbwidgets.core.pse.parameters - ImportError: Dask dependency is not included, so this functionality won't be available
2025-04-07 01:29:42,584 - WARNING - tvb.basic.readers - File 'hemispheres' not found in ZIP.
